# 設定

In [5]:
import os
from dotenv import load_dotenv
import time
load_dotenv()
COINCATCH_API_KEY = os.getenv("COINCATCH_API_KEY")
COINCATCH_SECRET_KEY = os.getenv("COINCATCH_SECRET_KEY")
COINCATCH_API_PASSPHRASE = os.getenv("COINCATCH_API_PASSPHRASE")

In [6]:
import requests
import time
import hmac
import hashlib
import base64

# === API URL ===
url = "https://api.coincatch.com/api/mix/v1/account/accounts?productType=umcbl"

# === 產生時間戳 ===
timestamp = str(int(time.time() * 1000))

# === 建立簽名 ===
# 簽名規則通常為：sign = HMAC_SHA256(secret, timestamp + method + requestPath + body)
# 因為這是 GET 請求且沒有 body
message = timestamp + "GET" + "/api/mix/v1/account/accounts?productType=umcbl"
signature = base64.b64encode(
    hmac.new(COINCATCH_SECRET_KEY.encode("utf-8"), message.encode("utf-8"), hashlib.sha256).digest()
).decode()

# === 設定 Header ===
headers = {
    "ACCESS-KEY": COINCATCH_API_KEY,
    "ACCESS-SIGN": signature,
    "ACCESS-TIMESTAMP": timestamp,
    "ACCESS-PASSPHRASE": COINCATCH_API_PASSPHRASE,
    "locale": "en-US",
    "Content-Type": "application/json"
}

# === 發送請求 ===
response = requests.get(url, headers=headers)

# === 處理回傳 ===
data = response.json()
if data.get("code") == "00000":
    account = data["data"][0]
    print("查詢成功 ✅")
    print(f"幣種: {account['marginCoin']}")
    print(f"可用餘額: {account['available']}")
    print(f"總資產(USDT): {account['usdtEquity']}")
else:
    print("查詢失敗 ❌")
    print(data)


查詢成功 ✅
幣種: USDT
可用餘額: 0
總資產(USDT): 0
